In [1]:
import pandas as pd
from tensorflow import keras
from os import path
import pickle


In [2]:
from proteinbert.finetuning import encode_train_and_valid_sets, encode_dataset
from proteinbert import OutputType, OutputSpec, evaluate_by_len, load_pretrained_model

In [3]:
from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, \
finetune, evaluate_by_len

from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [4]:
import wandb
from wandb.keras import WandbCallback

In [5]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

In [6]:
DATA_DIR = "../../data/"

In [7]:
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

In [8]:
pretrained_model_generator, input_encoder = load_pretrained_model("../../data/protein_bert/", "epoch_92400_sample_23500000.pkl")

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train_set = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_train.csv"), index_col=0)
test_set = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_test.csv"), index_col=0)
train_set["seq"] = train_set["heavy"] + train_set["light"]
test_set["seq"] = test_set["heavy"] + test_set["light"]

In [11]:
len(train_set)

1291

In [12]:
len(test_set)

260

In [13]:
learning_rate = 1e-4
patience = (6, 4)

In [14]:
sizes = [0.5, 0.6, 0.7, 0.8, 0.9]

In [16]:
def train_and_save_model(train_data, valid_data, test_data, size):
    wandb.init(project=f"Dataset size exp", entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
            get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]

    epoch_num = 100
    batch_size = 128
    #learning_rate = 1e-5
    wandb.config = {
          "learning_rate": learning_rate,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data["seq"], train_data["Y"], valid_data['seq'], valid_data["Y"], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)
    mod = model_generator.create_model(seq_len = 512)
    mod_name = f"2022_04_22_size{size}"
    mod.save(path.join(DATA_DIR, f"protein_bert/by_data_size/{mod_name}"))

    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    return confusion_matrix, f1

In [15]:
cms = {}
f1s = {}

In [17]:
for size in sizes:
    train, valid = train_test_split(train_set, test_size=1-size, random_state=42, stratify=train_set["Y"])
    #test = pd.concat([test, test_set])
    test = test_set
    #valid, test = train_test_split(test, test_size=0.5, random_state=333, stratify=test["Y"])
    print(len(train), len(valid), len(test))
    cm, f1_score = train_and_save_model(train, valid, test, size)
    cms[size] = cm
    f1s[size] = f1_score

645 646 260


wandb: Currently logged in as: kvetab (use `wandb login --relogin` to force relogin)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[2022_04_24-16:50:26] Training set: Filtered out 0 of 645 (0.0%) records of lengths exceeding 510.
[2022_04_24-16:50:27] Validation set: Filtered out 0 of 646 (0.0%) records of lengths exceeding 510.
[2022_04_24-16:50:27] Training with frozen pretrained layers...


2022-04-24 16:50:27.159831: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-24 16:50:27.728399: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9656 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:41:00.0, compute capability: 7.5
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2022-04-24 16:50:29.565892: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are en

Epoch 1/100


...

Epoch 2/100
10/10 [==============================] - 1s 110ms/step - loss: 0.5500 - val_loss: 0.5621
Epoch 3/100
10/10 [==============================] - 1s 109ms/step - loss: 0.4897 - val_loss: 0.4898
Epoch 4/100
10/10 [==============================] - 1s 111ms/step - loss: 0.4600 - val_loss: 0.4561
Epoch 5/100
10/10 [==============================] - 1s 108ms/step - loss: 0.4398 - val_loss: 0.4398
Epoch 6/100
10/10 [==============================] - 1s 111ms/step - loss: 0.4347 - val_loss: 0.4205
Epoch 7/100
10/10 [==============================] - 1s 112ms/step - loss: 0.4260 - val_loss: 0.4178
Epoch 8/100
10/10 [==============================] - 1s 111ms/step - loss: 0.4149 - val_loss: 0.4127
Epoch 9/100
10/10 [==============================] - 1s 113ms/step - loss: 0.4075 - val_loss: 0.4772
Epoch 10/100
10/10 [==============================] - 1s 107ms/step - loss: 0.4315 - val_loss: 0.4717
Epoch 11/100
10/10 [==============================] - 1s 108ms/step - loss: 0.4353 - val_l

10/10 [==============================] - 10s 465ms/step - loss: 0.3663 - val_loss: 0.3977
Epoch 2/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3775 - val_loss: 0.3766
Epoch 3/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3638 - val_loss: 0.3760
Epoch 4/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3658 - val_loss: 0.3703
Epoch 5/100
10/10 [==============================] - 3s 249ms/step - loss: 0.3539 - val_loss: 0.4184
Epoch 6/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3805 - val_loss: 0.3657
Epoch 7/100
10/10 [==============================] - 3s 253ms/step - loss: 0.3624 - val_loss: 0.3786
Epoch 8/100
10/10 [==============================] - 2s 248ms/step - loss: 0.3466 - val_loss: 0.3756
Epoch 9/100
10/10 [==============================] - 3s 252ms/step - loss: 0.3505 - val_loss: 0.3670
Epoch 10/100
10/10 [==============================] - 3s 253ms/step - loss: 0.3324 - val_loss: 0.3572


19/19 [==============================] - 13s 377ms/step - loss: 0.2923 - val_loss: 0.3448


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size0.9/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size0.9/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [18]:
size = 1
train = train_set
valid, test = train_test_split(test_set, test_size=0.5, random_state=333, stratify=test_set["Y"])
print(len(train_set), len(valid), len(test))
cm, f1_score = train_and_save_model(train, valid, test, size)
cms[size] = cm
f1s[size] = f1_score

1291 130 130


epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██▁▂▂▂▂▃▃▃▃▄▄▄▅▅▁
loss,█▅▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▃▂▂▂▂▂▂▂▂▁▁▂
lr,████████▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆▄▃▃▄▄▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▂▂▂▁▂▁▁▁▂▂▂▁
best_epoch,18
best_val_loss,0.32869
epoch,0
loss,0.2923
lr,1e-05
val_loss,0.34477


wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[2022_04_24-17:06:56] Training set: Filtered out 0 of 1291 (0.0%) records of lengths exceeding 510.
[2022_04_24-17:06:56] Validation set: Filtered out 0 of 130 (0.0%) records of lengths exceeding 510.
[2022_04_24-17:06:56] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


...

Epoch 2/100
11/11 [==============================] - 1s 110ms/step - loss: 0.5290 - val_loss: 0.4886
Epoch 3/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4669 - val_loss: 0.4886
Epoch 4/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4450 - val_loss: 0.5069
Epoch 5/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4493 - val_loss: 0.5267
Epoch 6/100
11/11 [==============================] - 1s 109ms/step - loss: 0.4278 - val_loss: 0.5224

Epoch 00006: ReduceLROnPlateau reducing learning rate to 0.0024999999441206455.
Epoch 7/100
11/11 [==============================] - 1s 108ms/step - loss: 0.4439 - val_loss: 0.4984
Epoch 8/100
11/11 [==============================] - 1s 111ms/step - loss: 0.4117 - val_loss: 0.4788
Epoch 9/100
11/11 [==============================] - 1s 107ms/step - loss: 0.4132 - val_loss: 0.4930
Epoch 10/100
11/11 [==============================] - 1s 111ms/step - loss: 0.4049 - val_loss: 0.4924
Epoch 11/

11/11 [==============================] - 10s 422ms/step - loss: 0.3829 - val_loss: 0.4600
Epoch 2/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3655 - val_loss: 0.4604
Epoch 3/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3596 - val_loss: 0.4616
Epoch 4/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3575 - val_loss: 0.4787
Epoch 5/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3616 - val_loss: 0.4599
Epoch 6/100
11/11 [==============================] - 3s 248ms/step - loss: 0.3555 - val_loss: 0.4684
Epoch 7/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3428 - val_loss: 0.4660
Epoch 8/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3364 - val_loss: 0.4654
Epoch 9/100
11/11 [==============================] - 3s 249ms/step - loss: 0.3249 - val_loss: 0.4736

Epoch 00009: ReduceLROnPlateau reducing learning rate to 2.499999936844688e-05.
Epoch 10/100
11/11 [=

21/21 [==============================] - 13s 362ms/step - loss: 0.3510 - val_loss: 0.4543


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size1/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/by_data_size/2022_04_22_size1/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [19]:
f1s

{0.5: 0.37894736842105264,
 0.6: 0.4864864864864865,
 0.7: 0.43010752688172044,
 0.8: 0.4807692307692308,
 0.9: 0.5094339622641509,
 1: 0.5531914893617021}

In [34]:
train_set

,Antibody_ID,heavy,light,Y,cluster,seq
535,2g60,EVQLQQSGGELAKPGASVKMSCKSSGYTFTAYAIHWAKQAAGAGLE...,DVLMTQAPLTLPVSLGDQASISCRSSQAIVHANGNTYLEWYLQKPG...,0,911,EVQLQQSGGELAKPGASVKMSCKSSGYTFTAYAIHWAKQAAGAGLE...
455,2a1w,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...,DVLMTQSPLSLPVSLGDQASISCRCSQSIVKSNGHTYLEWYLQKPG...,0,723,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...
459,2a77,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...,DVLMTQSPLSLPVSLGDQASISCRCSQSIVKSNGHTYLEWYLQKPG...,0,723,DVKLVESGGGLVKPGGSLRLSCAASGFTFRNYGMSWVRQTPEKRLE...
1120,4ffy,QVQLLQPGAELVKPGASMKLSCKASGYTFTNWWMHWVRLRPGRGLE...,NIVLTQSPASLAVSLGQRATISCRASESVDHYGNSFIYWYQQKPGQ...,0,478,QVQLLQPGAELVKPGASMKLSCKASGYTFTNWWMHWVRLRPGRGLE...
851,3l5x,EVTLKESGPVLVKPTETLTLTCTVSGFSLSTYGMGVGWIRQPPGKA...,EIVLTQSPATLSLSPGERATLSCRASKSISKYLAWYQQKPGQAPRL...,0,433,EVTLKESGPVLVKPTETLTLTCTVSGFSLSTYGMGVGWIRQPPGKA...
...,...,...,...,...,...,...
1664,5f9w,QVQLVQSGAEVKKPGASVTVSCQASGYTFTNYYVHWVRQAPGQGLQ...,EIVLTQSPATLSVSPGERATLSCRASQSVRSNLAWYQQRPGQAPRL...,0,271,QVQLVQSGAEVKKPGASVTVSCQASGYTFTNYYVHWVRQAPGQGLQ...
2017,5x5x,QVKLQQSGAEFVKAGASVKLSCKTSGYTFNNYWIHWVKQSPGQGLE...,DIELTQSPLSLPVSLGDQASISCTSSQSLLHSNGDTYLHWYLQKPG...,0,861,QVKLQQSGAEFVKAGASVKLSCKTSGYTFNNYWIHWVKQSPGQGLE...
1400,4qww,EVQLVESGGGLVQPKGSLKLSCAASGFTFNTYAMHWVRQAPGKGLE...,QIVLTQSPAIMSASPGEKVTMTCSASSSVSYMYWYHQKPGSSPKPW...,0,436,EVQLVESGGGLVQPKGSLKLSCAASGFTFNTYAMHWVRQAPGKGLE...
59,1cgs,RVQLLESGAELMKPGASVQISCKATGYTFSEYWIEWVKERPGHGLE...,ELVMTQSPLSLPVSLGDQASISCRPSQSLVHSNGNTYLHWYLQKPG...,0,103,RVQLLESGAELMKPGASVQISCKATGYTFSEYWIEWVKERPGHGLE...


# CV on all data

In [26]:
chen_data = pd.read_csv(path.join(DATA_DIR, "chen/deduplicated/chen_data_w_clusters.csv"), index_col=0)
chen_data.head()

,Antibody_ID,heavy,light,Y,cluster
0,12e8,EVQLQQSGAEVVRSGASVKLSCTASGFNIKDYYIHWVKQRPEKGLE...,DIVMTQSQKFMSTSVGDRVSITCKASQNVGTAVAWYQQKPGQSPKL...,0,677
1,15c8,EVQLQQSGAELVKPGASVKLSCTASGFNIKDTYMHWVKQKPEQGLE...,DIVLTQSPAIMSASLGERVTMTCTASSSVSSSNLHWYQQKPGSSPK...,0,685
2,1a0q,EVQLQESDAELVKPGASVKISCKASGYTFTDHVIHWVKQKPEQGLE...,DIELTQSPSSLSASLGGKVTITCKASQDIKKYIGWYQHKPGKQPRL...,1,102
3,1a14,QVQLQQSGAELVKPGASVRMSCKASGYTFTNYNMYWVKQSPGQGLE...,DIELTQTTSSLSASLGDRVTISCRASQDISNYLNWYQQNPDGTVKL...,0,442
4,1a2y,QVQLQESGPGLVAPSQSLSITCTVSGFSLTGYGVNWVRQPPGKGLE...,DIVLTQSPASLSASVGETVTITCRASGNIHNYLAWYQQKQGKSPQL...,0,59


In [43]:
chen_data["cluster"].value_counts()

18     59
24     35
28     28
8      25
7      21
       ..
588     1
562     1
786     1
722     1
329     1
Name: cluster, Length: 932, dtype: int64

In [44]:
def split_into_k_sets(k, data):
    total = len(data)
    size = total // k + 1
    clusters_by_size = data["cluster"].value_counts().index
    cluster_sizes = data["cluster"].value_counts()
    groups = { i: [] for i in range(k) }
    group = 0
    for clust in clusters_by_size:
        start_group = group
        if len(groups[group]) + cluster_sizes[clust] > size:
            group += 1
            group = group % k
        while len(groups[group]) + cluster_sizes[clust] > size and group != start_group:
            group += 1
            group = group % k
        if len(groups[group]) < size:
            groups[group] += list(data[data["cluster"] == clust].index)
        else:
            print("error")
    return groups

In [45]:
g = split_into_k_sets(10, chen_data)
for key, gr in g.items():
    print(len(gr))

152
154
155
154
156
156
156
156
156
156


In [65]:
def merge_clusters(df_in):
    df = df_in.copy()
    df["cluster_merged"] = df["cluster"]
    df["cluster_merged"][df["cluster"] < 300] = df["cluster"][df["cluster"] < 300] // 30
    df["cluster_merged"][df["cluster"] >= 300] = df["cluster"][df["cluster"] >= 300] // 100
    print(f'Unique clusters after merge: {df["cluster_merged"].nunique()}')
    return df

In [76]:
merged = merge_clusters(chen_data[~chen_data["cluster"].isin([18, 24, 28])])

Unique clusters after merge: 10


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [77]:
merged["cluster_merged"].value_counts()

3    214
0    198
4    164
7    161
6    159
5    158
8    135
1    104
2     71
9     65
Name: cluster_merged, dtype: int64

In [78]:
rest = chen_data[chen_data["cluster"].isin([18, 24, 28])].copy()
rest

,Antibody_ID,heavy,light,Y,cluster
82,1dqj,EVQLQESGPSLVKPSQTLSLTCSVTGDSVTSDYWSWIRKFPGNKLE...,DIVLTQSPATLSVTPGDSVSLSCRASQSISNNLHWYQQKSHESPRL...,1,28
123,1fvd,EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKL...,1,24
124,1fve,EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKL...,1,24
139,1gpo,EVKLQESGPSLVKPSQTLSLTCSVTGDSITSDFWSWIRQFPGNRLE...,DIELTQSPATLSVTPGNSVSISCRASQSLVNEDGNTYLFWYQQKSH...,1,28
163,1ic4,DVQLQESGPSLVKPSQTLSLTCSVTGDSITSAYWSWIRKFPGNRLE...,DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRL...,0,28
...,...,...,...,...,...
2347,6my5,EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTWIHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDIPRRISGYVAWYQQKPGK...,0,18
2368,6o39,EVQLVESGGGLVQPGGSLRLSCAASGFNIHSSSIHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQSVSSAVAWYQQKPGKAPKL...,0,24
2369,6o3a,EVQLVESGGGLVQPGGSLRLSCAASGFNFSSSSIHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQSVSSAVAWYQQKPGKAPKL...,0,24
2370,6o3b,EVQLVESGGGLVQPGGSLRLSCAASGFNIYYYSMHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQSVSSAVAWYQQKPGKAPKL...,0,24


In [69]:
assignments = {
    18: 9,
    24: 2,
    28: 1
}

def assign_rest(cluster):
    return assignments[cluster]

In [79]:
rest["cluster_merged"] = rest["cluster"].apply(assign_rest)
rest.head()

,Antibody_ID,heavy,light,Y,cluster,cluster_merged
82,1dqj,EVQLQESGPSLVKPSQTLSLTCSVTGDSVTSDYWSWIRKFPGNKLE...,DIVLTQSPATLSVTPGDSVSLSCRASQSISNNLHWYQQKSHESPRL...,1,28,1
123,1fvd,EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKL...,1,24,2
124,1fve,EVQLVESGGGLVQPGGSLRLSCAASGFNIKDTYIHWVRQAPGKGLE...,DIQMTQSPSSLSASVGDRVTITCRASQDVNTAVAWYQQKPGKAPKL...,1,24,2
139,1gpo,EVKLQESGPSLVKPSQTLSLTCSVTGDSITSDFWSWIRQFPGNRLE...,DIELTQSPATLSVTPGNSVSISCRASQSLVNEDGNTYLFWYQQKSH...,1,28,1
163,1ic4,DVQLQESGPSLVKPSQTLSLTCSVTGDSITSAYWSWIRKFPGNRLE...,DIVLTQSPATLSVTPGNSVSLSCRASQSIGNNLHWYQQKSHESPRL...,0,28,1


In [80]:
merged = pd.concat([merged, rest])
merged["cluster_merged"].value_counts()

3    214
0    198
4    164
7    161
6    159
5    158
8    135
1    132
9    124
2    106
Name: cluster_merged, dtype: int64

In [82]:
merged["seq"] = merged["heavy"] + merged["light"]

In [20]:
def train_and_save_named_model(train_data, valid_data, test_data, name, project_name):
    wandb.init(project=project_name, entity="kvetab")
    model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function = \
            get_model_with_hidden_layers_as_outputs, dropout_rate = 0.5)

    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience = patience[1], factor = 0.25, min_lr = 1e-07, verbose = 1),
        keras.callbacks.EarlyStopping(patience = patience[0], restore_best_weights = True),
        WandbCallback()
    ]

    epoch_num = 100
    batch_size = 128
    #learning_rate = 1e-5
    wandb.config = {
          "learning_rate": learning_rate,
          "epochs": epoch_num * 2,
          "batch_size": batch_size
        }
    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_data["seq"], train_data["Y"], valid_data['seq'], valid_data["Y"], \
            seq_len = 512, batch_size = batch_size, max_epochs_per_stage = epoch_num, lr = learning_rate, begin_with_frozen_pretrained_layers = True, \
            lr_with_frozen_pretrained_layers = 1e-02, n_final_epochs = 1, final_seq_len = 1024, final_lr = learning_rate / 10, callbacks = training_callbacks)
    mod = model_generator.create_model(seq_len = 512)
    mod.save(path.join(DATA_DIR, f"protein_bert/{name}"))

    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_data['seq'], test_data['Y'], \
            start_seq_len = 512, start_batch_size = 32)
    fn_fp = confusion_matrix.loc["0"][1] + confusion_matrix.loc["1"][0]
    f1 = confusion_matrix.loc["1"][1] / (confusion_matrix.loc["1"][1] + 0.5 * fn_fp)
    return confusion_matrix, f1

In [21]:
cms = {}
f1s = {}

In [87]:
for i in range(10):
    train = merged[merged["cluster_merged"] != i]
    test = merged[merged["cluster_merged"] == i]
    train, valid = train_test_split(train, test_size=0.2, random_state=333)
    cm, f1 = train_and_save_named_model(train, valid, test, f"10-fold-cv/2022_04_22_split_{i}", "10_fold_cv")
    cms[i] = cm
    f1s[i] = f1

wandb: wandb version 0.12.15 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


[2022_04_22-15:58:00] Training set: Filtered out 0 of 1082 (0.0%) records of lengths exceeding 510.
[2022_04_22-15:58:00] Validation set: Filtered out 0 of 271 (0.0%) records of lengths exceeding 510.
[2022_04_22-15:58:00] Training with frozen pretrained layers...


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/100
9/9 [==============================] - 11s 367ms/step - loss: 0.8908 - val_loss: 0.7102


/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
wandb: ERROR Can't save model, h5py returned error: Layer GlobalAttention has arguments in `__init__` and therefore must override `get_config`.


...

Epoch 2/100
9/9 [==============================] - 1s 133ms/step - loss: 0.5977 - val_loss: 0.5570
Epoch 3/100
9/9 [==============================] - 1s 133ms/step - loss: 0.5389 - val_loss: 0.4483
Epoch 4/100
9/9 [==============================] - 1s 132ms/step - loss: 0.5144 - val_loss: 0.4356
Epoch 5/100
9/9 [==============================] - 1s 134ms/step - loss: 0.4716 - val_loss: 0.4283
Epoch 6/100
9/9 [==============================] - 1s 133ms/step - loss: 0.4546 - val_loss: 0.4255
Epoch 7/100
9/9 [==============================] - 1s 132ms/step - loss: 0.4285 - val_loss: 0.4103
Epoch 8/100
9/9 [==============================] - 1s 132ms/step - loss: 0.4180 - val_loss: 0.4104
Epoch 9/100
9/9 [==============================] - 1s 132ms/step - loss: 0.4059 - val_loss: 0.4189
Epoch 10/100
9/9 [==============================] - 1s 133ms/step - loss: 0.4178 - val_loss: 0.4018
Epoch 11/100
9/9 [==============================] - 1s 133ms/step - loss: 0.3989 - val_loss: 0.3980
Epoch 12

9/9 [==============================] - 11s 511ms/step - loss: 0.3761 - val_loss: 0.3794
Epoch 2/100
9/9 [==============================] - 3s 286ms/step - loss: 0.3649 - val_loss: 0.3875
Epoch 3/100
9/9 [==============================] - 3s 288ms/step - loss: 0.3512 - val_loss: 0.3911
Epoch 4/100
9/9 [==============================] - 3s 287ms/step - loss: 0.3385 - val_loss: 0.3672
Epoch 5/100
9/9 [==============================] - 3s 285ms/step - loss: 0.3356 - val_loss: 0.3860
Epoch 6/100
9/9 [==============================] - 3s 286ms/step - loss: 0.3247 - val_loss: 0.3755
Epoch 7/100
9/9 [==============================] - 3s 286ms/step - loss: 0.3310 - val_loss: 0.3649
Epoch 8/100
9/9 [==============================] - 3s 287ms/step - loss: 0.3289 - val_loss: 0.3650
Epoch 9/100
9/9 [==============================] - 3s 287ms/step - loss: 0.3120 - val_loss: 0.3750
Epoch 10/100
9/9 [==============================] - 3s 297ms/step - loss: 0.3168 - val_loss: 0.3612
Epoch 11/100
9/9 [==

18/18 [==============================] - 13s 382ms/step - loss: 0.3279 - val_loss: 0.3513


INFO:tensorflow:Assets written to: ../../data/protein_bert/10-fold-cv/2022_04_22_split_9/assets


INFO:tensorflow:Assets written to: ../../data/protein_bert/10-fold-cv/2022_04_22_split_9/assets
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
/home/brazdilv/.conda/envs/ml/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [88]:
f1s

{0: 0.3870967741935484,
 1: 0.4375,
 2: 0.4,
 3: 0.5789473684210527,
 4: 0.19230769230769232,
 5: 0.2978723404255319,
 6: 0.4827586206896552,
 7: 0.5245901639344263,
 8: 0.5277777777777778,
 9: 0.5957446808510638}

In [23]:
from statistics import mean

In [24]:
f1_dict = {0: 0.3870967741935484,
 1: 0.4375,
 2: 0.4,
 3: 0.5789473684210527,
 4: 0.19230769230769232,
 5: 0.2978723404255319,
 6: 0.4827586206896552,
 7: 0.5245901639344263,
 8: 0.5277777777777778,
 9: 0.5957446808510638}

f1_scores = [value for key, value in f1_dict.items()]

In [25]:
mean(f1_scores)

0.44245954186007486